In [ ]:
using DataFrames, CSV, StatsPlots, Plots.Measures, Statistics, LaTeXStrings

# Handling data from the TI impurity simulations
In this notebook we want to show how the data files generated with our simulations for the two-dimensional Topological insulator with impurities can be handled to extract the desired information.

The simulation saves several .csv files, specifically it creates a folder named with the date and within that folder it creates 
* a log file with the parameters used for the simulation in the first line and several timestamped lines with the progress of the simulation for monitoring during running the code
* a number of .csv files that each contain the results of the simulation for one specific random impurity distribution

## The .log file

First we take a look at the log file.
For this purpose we read the first line of the .log from the data folder (small sample of an actual results folder).

In [ ]:
params = readline("data/2020-12-12/2020-12-12.log")

Most relevant for us will be the biggest Fourier coefficient $n_{max}$, so we store it in a variable to use later.

In [ ]:
nmax = parse(Int, first(split(last(split(params,"nmax="))," ,")))

We see the time stamp when the simulation started and with what parameters.
Note that one folder is created for each set of parameters, so this line can be used to automate the extraction of parameters and looping over several folders can be used to compare between parameters.

If we look at the second line we see that it logged that 10 runs where completed.
This is logged purely for information and monitoring purposes during the runs (the runs on a powerful desktop computer ran depending on parameters between several minutes to several days).

In [ ]:
readlines("data/2020-12-12/2020-12-12.log")[2]

## The .csv files

The .csv files contain the results of the simulation for one specific impurity configuration.
More specifically it contains

* the energy grid that will be the x-axis of most plots
* all calculated Green's function components that appear in the Floquet scattering matrix of the problem
* the Density of states at the impurity closest to the center
* the Density of states at the center of the impurity region
* the transmission from the left to the right end of the impurity region (T_RL)
* the transmission from the right to the left end of the impurity region (T_LR)

In principle the transmissions and density of states can be calculated from the Green's function components, but for convenience this has already been done.

Now let's load up one of the .csv files.
Note, that the Green's function components are complex numbers, so we specify the types for the columns.
The first two columns are going to be Floats, followed by $6 \cdot (2 n_{max}+1)$ ComplexFloats (6 for components between left and right, right and left, left and left, right and right, two times from center to center (spin) and $(2 n_{max}+1)$ for $n_{max}$ Fourier components positive/negative and zero) and the remaining three columns are Floats again.

In [ ]:
df=DataFrame(CSV.File("data/2020-12-12/2020-12-12_1.csv",types=[Float64, Float64, [Complex{Float64} for i=1:(2*nmax+1)*6]...,Float64, Float64, Float64]))
first(df,5) # show only first 5 rows

# Impurity averaging
We can now read the data for one specific impurity configuration.
Since we want to do impurity averaging we need to read the other files in the folder and in this case we will append them all to the same DataFrame and add a series index for each series.
We will use the number appended to the filename to number the configurations.

In [ ]:
files = filter(x -> x[end-3:end] == ".csv", readdir("data/2020-12-12/")) # create vector containing the filenames
data = DataFrame() # initialize empty dataframe
for i in files # loop over files
    # prepare dataframe
    df = DataFrame(CSV.File("data/2020-12-12/"*i,types=[Float64, Float64, [Complex{Float64} for i=1:(2*nmax+1)*6]...,Float64, Float64, Float64])) # read the .csv as dataframe
    s = parse(Int,last(split(first(split(i,'.')),'_'))) # extract series number from filename
    df[!, :series] = [s for k in 1:size(df)[1]] # label series in dataframe, could also add potential or other things
    # append the prepared df to main dataframe
    append!(data,df)
end

Our goal is now get the impurity averaged transmission $T_RL$ (from left to right) and also the density of states $DOS$ at the center depending on the energy $E$ scaled in units of the expected gap in the static case $\Delta$.
We can get that by simply grouping the data by the energy, i.e. get separate DataFrames for every energy and use the combine function to average and combine into the data we are interested in
(This follows the split-apply-combine logic, further reading [here](https://dataframes.juliadata.org/stable/man/split_apply_combine/)).

In [ ]:
gdf = groupby(data,:E)
result = combine(gdf,:T_RL => mean, :DOS => mean)
first(result,5) # show only first 5 rows

We can save the resulting dataframe as a .csv using the CSV package.

In [ ]:
CSV.write("impavg.csv",result)

## Plots
To plot the prepared data we can use the plot recipes that the StatPlots-Package provides

In [ ]:
plt1 = begin
    # plot reciept for initial plot
    @df result plot(:E,:T_RL_mean)
    # refine plot
    plot!(legend = false, xlabel = L"E/\Delta", ylabel = L"T_{RL}(E)")
    plot!(title = "Impurity averaged transmission")
end
plt2 = begin
    # plot reciept for initial plot
    @df result plot(:E,:DOS_mean)
    # refine plot
    plot!(legend=false,xlabel=L"E/\Delta",ylabel=L"DOS(E)")
    plot!(title = "Impurity averaged density of states")
end
# composite plot with the two plots prepared above
plot(plt1, plt2, layout = (1,2), size = (900,300), left_margin = [5mm 5mm], bottom_margin = [5mm 5mm])

Now we may want to see the difference between the impurity averaged results and the non-averaged traces.
For that we can group the data by the series index we added earlier and plot those traces.

In [ ]:
plt1 = begin
    plot() # initialize plot canvas
    for i in groupby(data,:series) # loop over the series in our dataframe
        @df i plot!(:E,:DOS) 
    end 
    # refine plot
    plot!(legend=false,xlabel=L"E/\Delta",ylabel=L"DOS(E)")
    plot!(title = "Density of states")
end
plt2 = begin
    plot() # initialize plot canvas
    for i in groupby(data,:series) # loop over the series in our dataframe
        @df i plot!(:E,:T_RL) # add the series to the canvas
    end
    # refine plot
    plot!(legend=false,xlabel=L"E/\Delta",ylabel=L"T_{RL}(E)")
    plot!(title = "Transmission")
end
# composite plot with the two plots prepared above
plot(plt1, plt2, layout = (1,2), size = (900,300), left_margin = [5mm 5mm], bottom_margin = [5mm 5mm])

To understand the effect the impurity averaging has we can also plot a few non-averaged traces next to the averaged results.
For that we can make use of the layout macro for more complicated composite plots and also the series index we added to the data earlier.

In [ ]:
l = @layout([a{0.6w} [b{0.35w,0.3h}; c{0.35w,0.3h}; d{0.35w,0.3h}]]) # set the layout
# create 3 single distribution plots
single=[] # initialize containter for subplots
for i in 1:3 
    d = groupby(data,:series)[i] # group data by series
    push!(single,@df d plot(:E,:T_RL,legend = false, widen = false, ylims = (0.,0.8),yticks=[0.0,0.4])) # append plot to container
    # some layouting depending on final position in the plot
    if i == 1
        plot!(xticks=[], top_margin = 8mm)
    elseif i == 2
        plot!(xticks=[])
    else
        plot!(xlabel = L"E/\Delta")
    end
end
# impurity averaged plot
avg = @df result plot(:E,:T_RL_mean,legend = false, xlabel = L"E/\Delta", ylabel = L"T_{RL}(E)",left_margin = [5mm 5mm], title = "Averaged transmission along\n some single distribution traces")
# final plot
plot(avg,single...,layout = l,size=(900,400),bottom_margin = 5mm)